<a href="https://colab.research.google.com/github/gogamid/ml-notebooks/blob/main/sccl-experiments/ewc_avalanche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-1i20elqa
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-1i20elqa
  Resolved https://github.com/ContinualAI/avalanche.git to commit 39c097fde7953e1b4515efa42503421bcac4ba88
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.5 MB/s eta 0:00:00
     ━━━━

In [2]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import EWC

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

scenario = SplitMNIST(n_experiences=5)

model = SimpleMLP(
    num_classes=scenario.n_classes,
    hidden_size=512,
    hidden_layers=1,
    drop_rate=0
)

loggers=[InteractiveLogger(),TextLogger(open('logSplitMNIST.txt', 'a'))]
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc_split_mnist"))


eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=loggers
)
cl_strategy = EWC(
    model,
    SGD(model.parameters(), lr=0.001),
    CrossEntropyLoss(),
    ewc_lambda=1,
    mode="separate",
    decay_factor=None,
    train_mb_size=256,
    train_epochs=10,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience ", experience.current_experience)
    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))
wandb.finish()

100%|██████████| 9912422/9912422 [00:00<00:00, 244535676.86it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 35722705.34it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 187153719.50it/s]


...

100%|██████████| 4542/4542 [00:00<00:00, 7124356.31it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Starting experiment...
Start of experience  0
-- >> Start of training phase << --
100%|██████████| 48/48 [00:03<00:00, 14.97it/s]
Epoch 0 ended.
100%|██████████| 48/48 [00:01<00:00, 26.21it/s]
Epoch 1 ended.
100%|██████████| 48/48 [00:01<00:00, 26.11it/s]
Epoch 2 ended.
100%|██████████| 48/48 [00:01<00:00, 25.69it/s]
Epoch 3 ended.
100%|██████████| 48/48 [00:01<00:00, 27.42it/s]
Epoch 4 ended.
100%|██████████| 48/48 [00:02<00:00, 18.02it/s]
Epoch 5 ended.
100%|██████████| 48/48 [00:03<00:00, 14.81it/s]
Epoch 6 ended.
100%|██████████| 48/48 [00:02<00:00, 22.09it/s]
Epoch 7 ended.
100%|██████████| 48/48 [00:02<00:00, 19.74it/s]
Epoch 8 ended.
100%|██████████| 48/48 [00:02<00:00, 23.02it/s]
Epoch 9 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 16/16 [00:00<00:00, 36.18it/s]
> Eval on experience 0 (Task 0) from test stream end

In [ ]:
wandb.finish()

Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,█▁▁▁▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,▁█▂▁▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,▁▁█▃▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,▁▁▁█▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004,▁▁▁▁█
Top1_Acc_Stream/eval_phase/test_stream/Task000,▁▂▅█▃
TrainingExperience,▁▃▅▆█
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,0.0
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,0.0
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,0.00448
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,0.01393
